In [1]:
import pandas as pd
import numpy as np
import boto3
from sklearn.model_selection import train_test_split
import sagemaker
from sagemaker import Session
import io
import sagemaker.amazon.common as smac
import os
from sagemaker.amazon.amazon_estimator import get_image_uri

In [3]:
df=pd.read_csv('student_scores.csv')
df.head()

,Hours,Scores
0,2.5,21
1,5.1,47
2,3.2,27
3,8.5,75
4,3.5,30


In [27]:
from sklearn.model_selection import train_test_split

X = df[['Hours']]  # features
y = df['Scores']   # labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [28]:
import sagemaker
from sagemaker import get_execution_role
import boto3

session = sagemaker.Session()
bucket = session.default_bucket()  # or specify your bucket name
prefix = 'linear-regression-student'

train_input = session.upload_data('data/train.csv', bucket=bucket, key_prefix=f'{prefix}/train')
test_input = session.upload_data('data/test.csv', bucket=bucket, key_prefix=f'{prefix}/test')

print("Train uploaded to:", train_input)
print("Test uploaded to:", test_input)


Train uploaded to: s3://sagemaker-ap-south-1-127214186818/linear-regression-student/train/train.csv
Test uploaded to: s3://sagemaker-ap-south-1-127214186818/linear-regression-student/test/test.csv


In [29]:
from sagemaker.amazon.linear_learner import LinearLearner

role = get_execution_role()

linear = LinearLearner(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    predictor_type='regressor',
    output_path=f's3://{bucket}/{prefix}/output',
    sagemaker_session=session
)

train_record_set = linear.record_set(X_train.values.astype('float32'), labels=y_train.values.astype('float32'))

linear.fit(train_record_set)


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: linear-learner-2025-05-15-03-54-27-230


2025-05-15 03:54:28 Starting - Starting the training job...
2025-05-15 03:54:45 Starting - Preparing the instances for training...
2025-05-15 03:55:04 Downloading - Downloading input data...
2025-05-15 03:55:45 Downloading - Downloading the training image........
2025-05-15 03:57:21 Training - Training image download completed. Training in progress.
2025-05-15 03:57:21 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[05/15/2025 03:57:15 INFO 139911132104512] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'ini

In [30]:
predictor = linear.deploy(initial_instance_count=1, instance_type='ml.m5.large')


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: linear-learner-2025-05-15-04-01-08-863
INFO:sagemaker:Creating endpoint-config with name linear-learner-2025-05-15-04-01-08-863
INFO:sagemaker:Creating endpoint with name linear-learner-2025-05-15-04-01-08-863


---------!

In [31]:
import numpy as np

test_sample = np.array([[5]])  # e.g., 5 hours studied
result = predictor.predict(test_sample.astype('float32'))

print(result)


[label {
  key: "score"
  value {
    float32_tensor {
      values: 56.5773468
    }
  }
}
]
